*In this tutorial we will walk through the capabilities of the tool in depth.*

### Introduction 

'Sugartrail' was developed to make it easier and faster for researchers to explore connections between companies, persons and addresses within [Companies House](https://www.gov.uk/government/organisations/companies-house). Researchers can build networks of connected companies, persons and addresses based on a defined set of connectivity criteria and then visualise these connections through an [OpenStreetMaps interface](https://ipyleaflet.readthedocs.io/en/latest/index.html).

### Prerequisites

Sugartrail uses the [Companies House Public Data API](https://developer-specs.company-information.service.gov.uk/companies-house-public-data-api/reference) to gather data on connected companies, persons and addresses. To access this API you will need a key which you can aquire by registering a [user account](https://developer.company-information.service.gov.uk/get-started/). Once you've aquired the key, insert it below as the string value of `api.basic_auth.username`:

In [ ]:
from sugartrail import api, mapview, base
from ipywidgets import VBox, HBox
import pandas as pd

api.basic_auth.username = ""

Lets make a test request to validate everything works by attempting to get all the officers who work at [this company](https://find-and-update.company-information.service.gov.uk/company/12411673). 

In [ ]:
company_id = "12411673"
api.get_company_officers(company_id)

### Initialising Networks 

To create a network we start from a single company, person or address. Networks are build and stored with the `Network` class. Lets go ahead and create a new network:

In [ ]:
network = base.Network()

`Network` accepts either a company ID, officer ID or address string as the initial node. For example, [this company](https://find-and-update.company-information.service.gov.uk/company/12411673): `company_id` = "12411673"

If we wanted to search by address, then `address` = "513 Tong Street, Flat 5, Bradford, England, BD4 6NA"

![title](../assets/images/spy.png)

For [this officer](https://find-and-update.company-information.service.gov.uk/officers/6WODVBRaegvY3UvEhcQxg0OsPkc/appointments), `officer_id` = "6WODVBRaegvY3UvEhcQxg0OsPkc"

![title](../assets/images/scrooge.png)

Lets build the network from `company_id`: 

In [ ]:
network.company_id="11004735"

We could also just initialise the network by passing `company_id` as an input: 

In [ ]:
network = base.Network(company_id="11004735")

Data about companies, persons and addresses are stored in several attributes within the `Network` class. If we check the `company_ids` property, we will find the entry we just created:

In [ ]:
network.company_ids

Each company is represented by its unique ID (`company_id`), number of hops from the origin company (`n`) and the company, address or person it connects to. As we've only saved the origin company so far, there isn't any information on links or connected nodes. There are also attributes for storing officer ids (`officer_ids`) and (`addresses`) although they have no information in them yet:

In [ ]:
network.officer_ids

In [ ]:
network.addresses

### Building Networks

We can now build the network by performing hops that will find new company IDs, officer IDs and addresses connected to the entities already stored within the network. 

There are a finite number of ways that officers, companies and addresses can be connected within Companies House:

#### Companies 

1. Companies → Officers: companies have officers 
2. Companies → Addresses: companies have a history of registered addresses 
3. Companies → Addresses: companies have correspondence addresses for their persons of significant control (psc)

#### Officers 

4. Officers → Companies: officers have appointments (companies they have a role in) 
5. Officers → Addresses: officers have correspondence addresses
6. Officers → Officers: officers may have duplicate enteries within Companies House; other officers using the same name and birth date (but different values for `officer_id`

#### Addresses 

7. Addresses → Officers: addresses are used as officer correspondence addresses 
8. Addresses → Companies: addresses are used as company correspondence addresses 

To build the network we can use any combination of this connectivity criteria. The above connections are implemented as methods that get called everytime we perform a hop: 

1. get_company_officers
2. get_company_address_history
3. get_psc_correspondance_address
3. get_officer_appointments
4. get_officer_correspondance_address 
5. get_officer_duplicates 
6. get_officers_at_address
7. get_companies_at_address

We can toggle each of these methods via boolean properties of the `Hop` subclass:

In [ ]:
network.hop.__dict__

We can see the `Hop` subclass contains all of the connections mentioned above set to `True` by default, therefore everytime we perform a hop, the network will use these methods to get data.

We also notice that there are some properties setting a "maxsize" limit. These properties ensure that if the number of results returned by the method exceeds this limit then the results will not be stored within the `Network` class properties. This limit is quite important when building networks as some of these methods can return 1000s of results and if we're not interested in these results they can make it difficult to visualise meaningful connections within the network (see Tutorial 3 for more on this). 

Lets go ahead and perform one hop using these default settings and see what addresses, companies and officers are added:

In [ ]:
network.perform_hop(1)

Lets now check out `company_ids`, `officer_ids` and `addresses` to see what new enteries have been added. Nothing new in `company_ids` but this is expected as none of the API methods above connect companies with companies in one hop:

In [ ]:
pd.DataFrame(network.company_ids)

We can see we now have an officer below in `officer_ids` and some of the other properties in the table now have values other than None. `node_type` describes what the type of node the company is connected to (Company, Person or Address), `node_id` provides the unique id for the node (`company_id`, `officer_id` or `address`) and `link_type` describes the relationship between the company and the node.

In [ ]:
pd.DataFrame(network.officer_ids)

We can interpret the table above as:

There is an officer with ID=`Nd2URspq4bvLy-hwzDZ0_p7FGJw` who is an officer to a company with ID=`11004735`. 

In [ ]:
pd.DataFrame(network.addresses)

We can see from the table above that:

`3rd Floor 13 Charles Ii Street London SW1Y 4QU England` is an address that used to be home to a company (with ID=`11004735`):

For reproducibility, each time we perform a hop, the methods and limit configs are stored in 

In [ ]:
pd.DataFrame(network.hop_history)

Lets perform another two hops: 

In [ ]:
network.perform_hop(2)

Now we can go ahead and visualise this in a map. To do this we need to get a bit more info that isn't present, namely the coordinates for all the addresses mentioned and the company names for each company. We can get this information via `run_map_preprocessing()`:

In [ ]:
network.run_map_preprocessing()

To see the information added, we can check out `address_history` and `companies` properties of our class:

In [ ]:
pd.DataFrame(network.address_history)

In [ ]:
pd.DataFrame(network.companies)

We can now visualise all the companies in the network with a UK address through OpenStreetMaps:

In [ ]:
map_data, path_table = mapview.build_map(network) 
hbox = HBox([path_table])
vbox = VBox([map_data, hbox])
vbox

Each marker represents a company in the network. Green markers represent active companies based at the address, red markers represent active companies no longer based at the address and black markers represent dissolved companies once based at the address. 

Select a marker to display additional information: 
- pop-up with the selected company's name and address
- table containing the most efficient paths from the origin to the selected company
- antpaths for each company in the network. Red antpath represents the path through all the historic addresses for the selected company. Black antpath represents the path from the network origin through all the addresses in the path to the selected company as displayed in the table.  

To read paths from the table we start from the bottom of the table where we find one or several rows containing our selected company (`Node`) but with differing values for `Node Index`, `Node Type` and `Link`. If we encounter multiple rows containing our selected node, this tells us there are multiple paths of equal length from the selected node (origin) to the origin. For example, consider the following table:  

<img src="../assets/images/kingdom_table.png" alt="Drawing" style="width: 700px;"/>


Pick N Mix London Limited (e) is a 'company at address' for 3rd Floor 13 Charles Ii Street (c) which is a 'historic address' for Kingdom of Sweets Ltd (a).

Additionally, Pick N Mix London Limited (d) is an appointment of (b) who is an officer of Kingdom of Sweets Ltd (a). 

### Network Persistance

The network object can be saved to `../assets/networks/` as json file:

In [ ]:
network.save('kingdom_of_sweets_network.json')

We can load the network by creating a new network and passing the filename: 

In [ ]:
network = base.Network(file='kingdom_of_sweets_network.json')